In [1]:
##Borrar datos del entorno
rm(list=ls())

In [2]:
##Crear folders de almacenamiento
dir.create("microdatos", showWarnings = F)

In [3]:
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr,janitor,survey)

Loading required package: pacman



In [4]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination="microdatos") {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}

In [55]:
#Uilizar la tabla de concentrado hogar
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip")

Descarga y descompresión exitosas para enigh2022_ns_concentradohogar_csv.zip



In [56]:
concentrado<-read.csv("microdatos/concentradohogar.csv")%>%
  janitor::clean_names()%>%
    #Crear deciles de hogares por ingreso
    mutate(deciles=ntile(ing_cor,10))

In [57]:
#Diseño muestral

dm<-concentrado%>%
  as_survey(weights=factor, strata=est_dis, ids=upm)

In [58]:
#Deciles de hogares por ingreso
dm%>%
  group_by(deciles)%>%
  summarise(hogares=survey_total(),
            gasto=survey_total(gasto_mon),
    vivienda=survey_total(vivienda))%>%
    #Quitar lo que termine con _se
    select(-ends_with("_se"))%>%
#Dejar los primeros cinco deciles
    filter(deciles<=5)%>%
    janitor::adorn_totals("row")%>%
    #Porcentajes
    mutate(pct_vivi=vivienda/gasto*100)

deciles,hogares,gasto,vivienda,pct_vivi
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,3491552,50842051082,5543902918,10.90417
2,3633525,68296919609,7489329172,10.96584
3,3615465,83040523764,9130705048,10.99548
4,3716750,99477127242,10552076603,10.60754
5,3745395,112733035629,11720093975,10.39633
Total,18202687,414389657327,44436107716,10.72327
